In [1]:
import os
import cv2
import math
import datetime
import numpy as np
from PIL import Image
from tqdm import tqdm
from enum import Enum
from patchify import patchify
from keras import backend as K
import matplotlib.pyplot as plt
from keras.layers import Rescaling
from keras.utils import to_categorical
from matplotlib.patches import Rectangle
from keras.models import Model, load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout

In [2]:
data_dir = r"E:\Segmentation\datasets\FloodNet-Supervised_v1.0"
batch_size = 32

In [3]:
print(f"Training data: {len(os.listdir(os.path.join(data_dir, 'train', 'train-org-img')))}")
print(f"Validation data: {len(os.listdir(os.path.join(data_dir, 'val', 'val-org-img')))}")
print(f"Test data: {len(os.listdir(os.path.join(data_dir, 'test', 'test-org-img')))}")

Training data: 1445
Validation data: 450
Test data: 448


In [30]:
def load_images_and_masks(data_dir, split="train"):
    image_list = []
    mask_list = []

    for file in tqdm(os.listdir(os.path.join(data_dir, f"{split}/{split}-org-img"))):
        if file.endswith(".jpg"): 
            image_path = os.path.join(data_dir,  f"{split}/{split}-org-img", file)
            mask_path = (os.path.join(data_dir,  f"{split}/{split}-label-img", file[:-4] + ".png"))

            image = cv2.imread(image_path)
            mask = cv2.imread(mask_path)

            image_list.append(image)
            mask_list.append(mask)

    return image_list, mask_list


# X_train, Y_train = load_images_and_masks(f'{data_dir}/', "train")
X_val, Y_val = load_images_and_masks(f'{data_dir}/', "val")
# X_test, Y_test = load_images_and_masks(f'{data_dir}/', "test")

# print(f"Training data: {len(X_train)}")
print(f"Validation data: {len(X_val)}")
# print(f"Test data: {len(X_test)}")

100%|██████████| 450/450 [03:48<00:00,  1.97it/s]

Validation data: 450


In [ ]:
class_map= {'Background':0, 'Building-flooded':1, 'Building-non-flooded':2, 'Road-flooded':3, 'Road-non-flooded':4, 'Water':5, 'Tree':6, 'Vehicle':7, 'Pool':8, 'Grass':9}
    
color_map = {'Background':[0, 0, 0], 'Building-flooded':[196, 0, 0], 'Building-non-flooded': [255,128,128], 'Road-flooded':[128, 128, 0],  'Road-non-flooded':[128, 128, 128], 
                 'Water': [0, 191, 255], 'Tree':[34, 139, 34], 'Vehicle': [123, 37, 118],  'Pool':[0, 68, 255],'Grass':[127, 224, 104]}

handles = [Rectangle((0,0),1,1, color = (np.array(c)/255)) for n,c in color_map.items()]

labels = [n for n,c in color_map.items()]


def to_RGB(label):
    """
    Suply our labale masks as input in RGB format. 
    Replace pixels with specific RGB values ...
    """    
    label_seg = np.zeros(label.shape,dtype=np.uint8)
    for key, val in class_map.items():
      label_seg [np.all(label == class_map[key],axis=-1)] = color_map[key]  
    return label_seg

def display_images_with_masks(image, mask):
    print("Image shape:", image.shape)
    print("Mask shape:", to_RGB(mask).shape)

    plt.figure(figsize=(16, 14))
    
    plt.subplot(131)
    plt.title('Testing Image')
    plt.imshow(image)

    plt.subplot(132)
    plt.title('Testing Label')
    plt.imshow(to_RGB(mask))

    plt.legend(handles, labels, bbox_to_anchor=(-0.8, -0.5), loc='lower center', ncol=5)
    plt.show()


X_val_array = np.array(X_val[10:])
Y_val_array = np.array(Y_val[10:])
display_images_with_masks(X_val_array, Y_val_array)